In [1]:
import pandas as pd

In [4]:
# Re-import after reset
import pandas as pd
from geopy.distance import geodesic
from datetime import datetime, timedelta

# Load GPS data
FILE_PATH = 'data/Gps-Collection.csv'  # Path to your Excel file
# Sample: Load GPS data
df = pd.read_csv(FILE_PATH)
df.columns = df.columns.str.strip()

# Step 1: Define shop locations (add more as needed)
shops = [
    {"name": "Shop A", "location": (6.13852, 80.10066), "radius": 30},  # lat, lon, radius in meters
    {"name": "Shop B", "location": (6.13820, 80.10100), "radius": 30},
]

# Step 2: Parse timestamps (assume 'DateTime' column exists and is in string format)
df['DateTime'] = pd.to_datetime(df['DateTime'], errors='coerce')
df = df.dropna(subset=['Latitude', 'Longitude', 'DateTime'])
gps_points = df[['Latitude', 'Longitude', 'DateTime']].values.tolist()

# Step 3: Detect check-in/check-out per shop
visit_records = []

for shop in shops:
    in_zone = False
    check_in_time = None
    for i, (lat, lon, ts) in enumerate(gps_points):
        distance = geodesic((lat, lon), shop['location']).meters
        if distance <= shop['radius']:
            if not in_zone:
                in_zone = True
                check_in_time = ts
        else:
            if in_zone:
                check_out_time = gps_points[i-1][2]
                duration = (check_out_time - check_in_time).total_seconds() / 60  # in minutes
                visit_records.append({
                    "shop": shop['name'],
                    "check_in": check_in_time,
                    "check_out": check_out_time,
                    "duration_min": round(duration, 2)
                })
                in_zone = False


C:\Users\HP\AppData\Local\Temp\ipykernel_16124\2958452617.py:19: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['DateTime'] = pd.to_datetime(df['DateTime'], errors='coerce')


In [5]:
visit_records

[{'shop': 'Shop A',
  'check_in': Timestamp('2025-04-29 16:46:45'),
  'check_out': Timestamp('2025-04-29 21:05:54'),
  'duration_min': 259.15},
 {'shop': 'Shop A',
  'check_in': Timestamp('2025-04-29 21:06:48'),
  'check_out': Timestamp('2025-04-29 21:27:03'),
  'duration_min': 20.25},
 {'shop': 'Shop A',
  'check_in': Timestamp('2025-04-29 21:29:16'),
  'check_out': Timestamp('2025-04-29 21:29:16'),
  'duration_min': 0.0},
 {'shop': 'Shop A',
  'check_in': Timestamp('2025-04-29 21:29:46'),
  'check_out': Timestamp('2025-04-29 21:29:46'),
  'duration_min': 0.0},
 {'shop': 'Shop A',
  'check_in': Timestamp('2025-04-29 21:29:59'),
  'check_out': Timestamp('2025-04-29 21:32:01'),
  'duration_min': 2.03},
 {'shop': 'Shop A',
  'check_in': Timestamp('2025-04-29 21:32:42'),
  'check_out': Timestamp('2025-04-29 22:22:23'),
  'duration_min': 49.68},
 {'shop': 'Shop A',
  'check_in': Timestamp('2025-04-29 22:25:06'),
  'check_out': Timestamp('2025-04-29 22:25:06'),
  'duration_min': 0.0},
 {'sh

In [2]:
import folium
from folium.plugins import TimestampedGeoJson
import pandas as pd
FILE_PATH = 'data/Gps-Collection.csv'  # Path to your Excel file
# Sample: Load GPS data
df = pd.read_csv(FILE_PATH)
df['DateTime'] = pd.to_datetime(df['DateTime'], errors='coerce')
df = df.dropna(subset=['Latitude', 'Longitude', 'DateTime'])

# Convert datetime to ISO 8601
df['TimeISO'] = df['DateTime'].dt.strftime('%Y-%m-%dT%H:%M:%SZ')

# Build features for TimestampedGeoJson
features = []
for i, row in df.iterrows():
    feature = {
        'type': 'Feature',
        'geometry': {
            'type': 'Point',
            'coordinates': [row['Longitude'], row['Latitude']],
        },
        'properties': {
            'time': row['TimeISO'],
            'popup': f"Time: {row['TimeISO']}",
            'icon': 'circle',
            'iconstyle': {
                'fillColor': 'blue',
                'fillOpacity': 0.6,
                'stroke': 'true',
                'radius': 5
            },
        },
    }
    features.append(feature)

# Create the map
m = folium.Map(location=[df['Latitude'].iloc[0], df['Longitude'].iloc[0]], zoom_start=16)

TimestampedGeoJson(
    {
        'type': 'FeatureCollection',
        'features': features,
    },
    period='PT1M',  # 1-minute interval
    add_last_point=True,
    auto_play=True,
    loop=False,
    max_speed=1,
    loop_button=True,
    date_options='YYYY-MM-DD HH:mm:ss',
    time_slider_drag_update=True,
).add_to(m)


C:\Users\HP\AppData\Local\Temp\ipykernel_16124\2581749229.py:7: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['DateTime'] = pd.to_datetime(df['DateTime'], errors='coerce')


In [3]:
m